# Administration: Create sign-up-link



NOTE: This adheres to the scheme in
`osparc-simcore/scripts/demo/templates/invitation-codes-{{ deploy  }}-{{ datestamp }}.md.jinja2`

## Config: Endpoints and Credentials

In [ ]:
numberOfInvitationLinks = 1

### Static configuration variables

In [ ]:
import os
osparcURL = 'https://' + os.environ.get('MACHINE_FQDN')
length_of_invitation_code_characters = 30
invitationAction = "INVITATION"
invitationIssuer = os.environ.get('SMTP_USERNAME') + "@" + os.environ.get('SMTP_HOST')

### Login Credentials of invitation creator (you)

In [ ]:
#######################
currentUsername = ''
currentPassword = ''

## Clone oSparc Repo and install python dependencies

In [ ]:
#%%capture
import sys
print("FYI: You can ignore even fatal errors in this section....")
!python3 -m pip install requests
!python3 -m pip install sqlalchemy
!python3 -m pip install psycopg2-binary

### Authentication with oSPARC

In [ ]:
import requests
import json
#######################
print("Logging in...")
s = requests.Session()
data = {
  "email": currentUsername,
  "password": currentPassword
}
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
r = s.post(osparcURL + "/v0/auth/login", data=json.dumps(data),headers=headers)
if r.status_code == 200:
    print("Success!")
else:
    raise ValueError("Authentication failed!")

## Fetching Information

### Fetching Information: oSPARC REST API

In [ ]:
import random
import string
import datetime
import sqlalchemy as db
import psycopg2
import os
import sys
import json
import importlib
from pathlib import Path
#######################
def createInvitationLink(r):
    r = s.get(osparcURL + "/v0/me")
    userID = int(r.json()['data']['id'])
    #print("UserID of invitation creator:",userID)
    random_string = "".join(random.choice(string.ascii_letters + string.digits) for i in range(length_of_invitation_code_characters))
    #print("Invitation random key:",random_string)
    dateTimeObj = datetime.datetime.now()
    timestampStr = dateTimeObj.strftime("%Y%m%d")
    #print("Invitation datestamp:",timestampStr)
                             
    #########################################
    #######################
    PG_PASSWORD = os.environ.get('POSTGRES_PASSWORD')
    PG_ENDPOINT=os.environ.get('POSTGRES_ENDPOINT')
    PG_DB=os.environ.get('POSTGRES_DB')
    PG_USER=os.environ.get('POSTGRES_USER')
    pgEngineURL= "postgresql://{user}:{password}@{host}:{port}/{database}".format(
            user=PG_USER,
            password=PG_PASSWORD,
            database=PG_DB,
            host=PG_ENDPOINT.split(":")[0],
            port=int(PG_ENDPOINT.split(":")[1]),
        )
    engine = db.create_engine(pgEngineURL)
    connection = engine.connect()
    metadata = db.MetaData()
    confirmations = db.Table('confirmations', metadata, autoload=True, autoload_with=engine)
    ##########
    # Automatically building datestamp iterator for current invite:
    query = db.select([confirmations]).where(confirmations.c.data.like("%" + timestampStr + "%"))
    ResultProxy = connection.execute(query)
    ResultSet = ResultProxy.fetchall()
    dataTimestamp = timestampStr + "-" + str(len(ResultSet) + 1)
    dataObj = json.dumps({"guest":"invitation-" + dataTimestamp, "issuer":invitationIssuer},indent=2)
    #Inserting new record into database
    query = db.insert(confirmations).values(code=random_string, user_id=userID, action=invitationAction, data=dataObj,created_at=dateTimeObj) 
    ResultProxy = connection.execute(query)
    print("INVITATION LINK:",osparcURL + "/#/registration/?invitation=" + random_string)
    return osparcURL + "/#/registration/?invitation=" + random_string

### Create Invitation Links

In [ ]:
listOfInvitations = []
for i in range(numberOfInvitationLinks):
    listOfInvitations.append(createInvitationLink(r))